In [2]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [4]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [6]:
from bs4 import BeautifulSoup
import requests
import re

In [7]:
from langchain_teddynote.messages import AgentCallbacks
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [8]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=5)

In [9]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [10]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

In [11]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [12]:
tools = [search_keyword, python_repl_tool, naver_news_crawl]

In [13]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

In [14]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [15]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0].tool}")
    print(f'===== 도구 호출 =====')

In [16]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [17]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [18]:
store = {}

In [19]:
def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴

In [20]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [21]:
response = agent_with_chat_history.stream(
    {'input': '안녕? 내 이름은 이인환이야.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [22]:
for step in response:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)    



> Entering new None chain...
안녕하세요, 이인환님! 어떻게 도와드릴까요?

> Finished chain.
===== 최종 답변 =====
안녕하세요, 이인환님! 어떻게 도와드릴까요?
===== 최종 답변 =====


In [23]:
response = agent_with_chat_history.stream(
    {'input': '내 이름이 뭐라고?'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

for step in response:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)    



> Entering new None chain...
이인환님이라고 하셨습니다. 맞나요?

> Finished chain.
===== 최종 답변 =====
이인환님이라고 하셨습니다. 맞나요?
===== 최종 답변 =====


In [24]:
response = agent_with_chat_history.stream(
    {'input': '랭체인에 대한 뉴스를 검색해주세요'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

for step in response:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)    



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====

Invoking: `search_keyword` with `{'query': '랭체인'}`


검색어: 랭체인
[{'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}, {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFAtSUUxVTY0M3FhSmtBZTd1TFN5ZzFtSEU1VDdxcjkyVHVZZHBRdmtyTkJQZlNtUS1xVmtwcTB4MEhWb05KWUUySXJFNmdZcXBqV2ZyVnVTb2ROeW50UkRMa2J6WEtFY2V2WXc?oc=5', 'content': "시스코·랭체인, AI 에이전트 '표준 통신 프로토콜' 만든다 - AI타임스"}, {'url': 'https://news.google.com/rss/articles/CBMiS0FVX3lxTE5PU0M1MVBaZTVFcE51S3ZLUHFlemJISHZZdmxSTVRPdDl6RTBUY2JURFYyUTlnclFMdE93UDRLbGlaOWtXWUJydlNsZw?oc=5', 'content': 'GUAVA AI 칼럼 : 뒤늦은 MCP 체험기 - 브런치스토리'}, {'url': 'https://news.google.com/rss/articles/CBMiwgJBVV95cUxPM0hHcjNGR0RiT0M2QlplRlcxdjdYZE5WTk9z